In [1]:
import numpy as np
import xgboost as xgb
import pandas as pd
import gc
from sklearn.metrics import accuracy_score,r2_score
import matplotlib.pyplot as plt
import time
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import datetime

SEED = 314159265
VALID_SIZE = 0.2
TARGET = 'outcome'

In [2]:
starttime = time.time()
# read in data
mainFrame=pd.read_csv('Small.csv',parse_dates=['date'])
#model = joblib.load(r'C:\Hedge Fund Project\training\modelv1.plk')
mainFrame.set_index(['entityID','date'],inplace=True)
#mainFrame=mainFrame20052018
mainFrame.sort_index(inplace=True)
targets=mainFrame.iloc[:,-7:]
features = mainFrame.iloc[:,:-7]
gc.collect()
endtime = time.time()
print("It takes {}s to load data".format(endtime-starttime))

It takes 18.218702793121338s to load data


In [3]:
maskTrain=(mainFrame.index.get_level_values(1)>='2005') & (mainFrame.index.get_level_values(1)<'2005-10-01')

maskTest=(mainFrame.index.get_level_values(1)>='2005-10-01') & (mainFrame.index.get_level_values(1)<'2006')

x_train=np.array(features[maskTrain])
y_train=np.array(targets['ztargetMedian5'][maskTrain])
x_train[np.isinf(x_train)]=100000000
y_train=y_train*1
y_train=y_train.astype(int)

x_test=np.array(features[maskTest])
y_test = np.array(targets['ztargetMedian5'][maskTest])
x_test[np.isinf(x_test)]=100000000
y_test=y_test*1
y_test=y_test.astype(int)

dtrain = xgb.DMatrix(x_train, label=y_train)
dvalid =xgb.DMatrix(x_test, label=y_test)
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2

In [4]:
def score(params):
    print("Training with params: ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm_model = xgb.train(params, dtrain, num_round,
                          evals=watchlist,
                          verbose_eval=True)
    predictions = gbm_model.predict(dvalid,
                                    ntree_limit=gbm_model.best_iteration + 1)
    
    predictions[predictions>0.5] = 1
    predictions[predictions<=0.5] = 0
    
    score = accuracy_score(y_test, predictions)
    # TODO: Add the importance for the selected features
    print("\tScore {0}\n\n".format(score))
    # The score function should return the loss (1-score)
    # since the optimize function looks for the minimum
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}

In [5]:
def optimize(
             #trials, 
             random_state=SEED):
    """
    This is the optimization function that given a space (space here) of 
    hyperparameters and a scoring function (score here), finds the best hyperparameters.
    """
    # To learn more about XGBoost parameters, head to this page: 
    # https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
    space = {
        'n_estimators': hp.quniform('n_estimators', 100, 1000, 1),
        'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
        # A problem with max_depth casted to float instead of int with
        # the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'eval_metric': 'auc',
        'objective': 'binary:logistic',
        # Increase this number if you have more cores. Otherwise, remove it and it will default 
        # to the maxium number. 
        'nthread': 4,
        'booster': 'gbtree',
        'tree_method': 'exact',
        'silent': 1,
        'seed': random_state
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest, 
                # trials=trials, 
                max_evals=250)
    return best

In [6]:
best_hyperparams = optimize(
                            #trials
                            )
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)
gc.collect()

Training with params:                                                                                                  
{'booster': 'gbtree', 'colsample_bytree': 0.8, 'eta': 0.125, 'eval_metric': 'auc', 'gamma': 0.6000000000000001, 'max_depth': 8, 'min_child_weight': 5.0, 'n_estimators': 302.0, 'nthread': 4, 'objective': 'binary:logistic', 'seed': 314159265, 'silent': 1, 'subsample': 0.6000000000000001, 'tree_method': 'exact'}
[0]	eval-auc:0.50348	train-auc:0.637718                                                                                

[1]	eval-auc:0.509878	train-auc:0.67076                                                                                

[2]	eval-auc:0.511186	train-auc:0.694134                                                                               

[3]	eval-auc:0.508671	train-auc:0.705042                                                                               

[4]	eval-auc:0.511252	train-auc:0.720217                                             

[64]	eval-auc:0.517634	train-auc:0.881969                                                                              

[65]	eval-auc:0.517569	train-auc:0.882702                                                                              

[66]	eval-auc:0.517911	train-auc:0.883077                                                                              

[67]	eval-auc:0.518056	train-auc:0.88323                                                                               

[68]	eval-auc:0.518195	train-auc:0.884331                                                                              

[69]	eval-auc:0.518044	train-auc:0.886956                                                                              

[70]	eval-auc:0.518043	train-auc:0.888441                                                                              

[71]	eval-auc:0.518981	train-auc:0.889483                                                                              

[72]	eval-auc:0.518989	train-auc

[131]	eval-auc:0.519331	train-auc:0.941297                                                                             

[132]	eval-auc:0.519569	train-auc:0.941927                                                                             

[133]	eval-auc:0.519699	train-auc:0.942342                                                                             

[134]	eval-auc:0.519676	train-auc:0.942652                                                                             

[135]	eval-auc:0.519664	train-auc:0.943201                                                                             

[136]	eval-auc:0.519512	train-auc:0.943689                                                                             

[137]	eval-auc:0.519623	train-auc:0.944136                                                                             

[138]	eval-auc:0.519559	train-auc:0.944471                                                                             

[139]	eval-auc:0.51965	train-auc

[198]	eval-auc:0.519724	train-auc:0.969139                                                                             

[199]	eval-auc:0.519706	train-auc:0.969324                                                                             

[200]	eval-auc:0.519675	train-auc:0.969802                                                                             

[201]	eval-auc:0.519568	train-auc:0.970431                                                                             

[202]	eval-auc:0.519431	train-auc:0.970686                                                                             

[203]	eval-auc:0.519314	train-auc:0.971051                                                                             

[204]	eval-auc:0.519399	train-auc:0.971212                                                                             

[205]	eval-auc:0.519571	train-auc:0.97142                                                                              

[206]	eval-auc:0.519766	train-au

[265]	eval-auc:0.520989	train-auc:0.984432                                                                             

[266]	eval-auc:0.520999	train-auc:0.984672                                                                             

[267]	eval-auc:0.520966	train-auc:0.984907                                                                             

[268]	eval-auc:0.520946	train-auc:0.985056                                                                             

[269]	eval-auc:0.52098	train-auc:0.985073                                                                              

[270]	eval-auc:0.521069	train-auc:0.985253                                                                             

[271]	eval-auc:0.520837	train-auc:0.985518                                                                             

[272]	eval-auc:0.520662	train-auc:0.985585                                                                             

[273]	eval-auc:0.520379	train-au

...

[259]	eval-auc:0.525406	train-auc:0.795434                                                                             

[260]	eval-auc:0.525483	train-auc:0.795721                                                                             

[261]	eval-auc:0.525194	train-auc:0.796185                                                                             

[262]	eval-auc:0.525206	train-auc:0.796382                                                                             

[263]	eval-auc:0.525271	train-auc:0.796557                                                                             

[264]	eval-auc:0.525263	train-auc:0.796966                                                                             

[265]	eval-auc:0.525214	train-auc:0.797153                                                                             

[266]	eval-auc:0.525149	train-auc:0.797681                                                                             

[267]	eval-auc:0.525018	train-au

[326]	eval-auc:0.523443	train-auc:0.817745                                                                             

[327]	eval-auc:0.523494	train-auc:0.817971                                                                             

[328]	eval-auc:0.52359	train-auc:0.818296                                                                              

[329]	eval-auc:0.523662	train-auc:0.818455                                                                             

[330]	eval-auc:0.523688	train-auc:0.818721                                                                             

[331]	eval-auc:0.52388	train-auc:0.819239                                                                              

[332]	eval-auc:0.524026	train-auc:0.819381                                                                             

[333]	eval-auc:0.52403	train-auc:0.819385                                                                              

[334]	eval-auc:0.524046	train-au

[393]	eval-auc:0.522857	train-auc:0.837439                                                                             

[394]	eval-auc:0.522804	train-auc:0.837632                                                                             

[395]	eval-auc:0.522863	train-auc:0.837858                                                                             

[396]	eval-auc:0.522851	train-auc:0.838277                                                                             

[397]	eval-auc:0.522724	train-auc:0.838377                                                                             

[398]	eval-auc:0.52276	train-auc:0.838651                                                                              

[399]	eval-auc:0.522791	train-auc:0.838678                                                                             

[400]	eval-auc:0.522728	train-auc:0.839099                                                                             

[401]	eval-auc:0.52266	train-auc

[460]	eval-auc:0.521833	train-auc:0.852535                                                                             

[461]	eval-auc:0.521796	train-auc:0.8527                                                                               

[462]	eval-auc:0.521769	train-auc:0.852897                                                                             

[463]	eval-auc:0.521923	train-auc:0.853234                                                                             

[464]	eval-auc:0.521917	train-auc:0.853318                                                                             

[465]	eval-auc:0.521842	train-auc:0.853664                                                                             

[466]	eval-auc:0.52191	train-auc:0.854072                                                                              

[467]	eval-auc:0.521991	train-auc:0.854355                                                                             

[468]	eval-auc:0.522018	train-au

KeyboardInterrupt: 